In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks')

In [0]:
import common_machine_learning as common
import pandas as pd
import numpy as np

In [0]:
train = common.cargar_set_optimizado('/content/drive/My Drive/Colab Notebooks/sets_de_datos/dima_train_fill_0.csv')

In [0]:
train['año'] = train['fecha'].dt.year
train['mes'] = train['fecha'].dt.month
train['dia'] = train['fecha'].dt.day

In [0]:
def metros_no_cubiertos(m_totales, m_cubiertos):
    if m_cubiertos >= m_totales:
        return 0
    return m_totales - m_cubiertos
    

train['metros_no_cubiertos'] =\
    train.apply(lambda x: metros_no_cubiertos(x['metrostotales'], x['metroscubiertos']), axis = 1)

In [19]:
train['metros_no_cubiertos'].mean()

14.069029166666667

In [0]:
def cantidad_espacios(df):
    cantidad_espacios = 0
    for col in ['habitaciones', 'garages', 'banos', 'gimnasio', 'usosmultiples', 'piscina']:
        cantidad_espacios += int(df[col])
    return cantidad_espacios

train['cantidad_espacios'] = train.apply(cantidad_espacios, axis = 1)

In [0]:
def metros_x_espacio(cantidad_espacios, metros_cubiertos):
    if cantidad_espacios <= 0 or metros_cubiertos <= 0:
        return 0
    return metros_cubiertos / cantidad_espacios

train['metros_x_espacio'] = train.apply(lambda x: metros_x_espacio(x['cantidad_espacios'], x['metroscubiertos']), axis = 1)

In [0]:
def metros_en_varios_espacios(repeticiones_espacio, metros_x_espacio):
    if repeticiones_espacio <= 0 or metros_x_espacio <= 0:
        return 0
    return repeticiones_espacio * metros_x_espacio

for col in ['habitaciones', 'garages', 'banos', 'gimnasio', 'usosmultiples', 'piscina']:
    train['metros_x_' + col] =\
        train.apply(lambda x: metros_en_varios_espacios(x[col], x['metros_x_espacio']), axis = 1)

In [0]:
train['cosas_cercanas'] = train['escuelascercanas'].astype(int)  + train['centroscomercialescercanos'].astype(int)

In [0]:
metros_x_tipodepropiedad = train.groupby(['tipodepropiedad']).agg({'metroscubiertos' : 'mean'}).to_dict()
train['metros_x_tipodepropiedad'] =\
  train.apply(lambda x: 
                metros_x_tipodepropiedad['metroscubiertos'].get(
                    x['tipodepropiedad'], 
                    train['metroscubiertos'].astype(np.float32).mean()
                    ), axis = 1)

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [0]:
TARGET = 'precio'
FEATURES = ['id', 'tipodepropiedad', 'ciudad', 'provincia', 'antiguedad', 'habitaciones', 
            'garages', 'banos', 'metroscubiertos', 'metrostotales', 'idzona', 'lat', 
            'lng', 'fecha', 'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
            'centroscomercialescercanos', 'precio', 'año', 'mes', 'dia', 
            'metros_no_cubiertos', 'cantidad_espacios', 'metros_x_espacio',
            'metros_x_habitaciones', 'metros_x_garages', 'metros_x_banos',
            'metros_x_gimnasio', 'metros_x_usosmultiples', 'metros_x_piscina',
            'cosas_cercanas']

In [0]:
def evaluar_modelo(modelo, X_test, y_test):
    y_pred = modelo.predict(X_test)
    errors = abs(y_pred - y_test)
    mape = 100 * np.mean(errors / y_test)
    accuracy = 100 - mape
    print('Performance del modelo:')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [0]:
params = {'max_depth': 51, 'max_features': 9, 'min_samples_leaf': 2, 'min_samples_split': 9, 'n_estimators': 236}
rf_regressor = RandomForestRegressor(**params)

In [0]:
rf.fit(train[FEATURES], target_train.values.ravel())

In [0]:
rf.predict(test[FEATURES])